## Code to visualise dataset + labels for debugging

In [8]:
import os
import random
import glob
from PIL import Image, ImageDraw, ImageFont

# === Pfade ===
image_dir = "../huggingface/ai_shelf/artificial_rotated_mult_back_advanced_pipe/images/train"
label_dir = "../huggingface/ai_shelf/artificial_rotated_mult_back_advanced_pipe/labels/train"
output_dir = "visualized_labels_old_dataset"
os.makedirs(output_dir, exist_ok=True)

# === ID → Klassenname Zuordnung ===
id_to_class = {
    0: "apple",
    1: "avocado",
    2: "banana",
    3: "coffee",
    4: "cucumber",
    5: "fruit tea",
    6: "lemon",
    7: "oatmeal",
    8: "pasta",
    9: "tomato sauce"
}

# === Alle Bildpfade sammeln & zufällig auswählen ===
all_image_paths = glob.glob(os.path.join(image_dir, "*.png"))
selected_images = random.sample(all_image_paths, 200)

# === Grid-Parameter ===
grid_size = 20
grid_rows = 4
grid_cols = 5

# === YOLO-Labels einzeichnen + Text anzeigen ===
def draw_yolo_boxes(image, label_path, color=(255, 0, 0)):
    draw = ImageDraw.Draw(image)
    W, H = image.size
    font = ImageFont.load_default()

    if not os.path.exists(label_path):
        return image  # kein Label vorhanden

    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue  # ungültige Zeile
            class_id, x, y, w, h = map(float, parts)
            class_id = int(class_id)
            class_name = id_to_class.get(class_id, str(class_id))

            x1 = (x - w / 2) * W
            y1 = (y - h / 2) * H
            x2 = (x + w / 2) * W
            y2 = (y + h / 2) * H

            draw.rectangle([x1, y1, x2, y2], outline=color, width=2)

            # Text-Hintergrund und Text
            text_bbox = draw.textbbox((x1, y1), class_name, font=font)
            text_bg = [text_bbox[0], text_bbox[1], text_bbox[2], text_bbox[3]]
            draw.rectangle(text_bg, fill=color)
            draw.text((x1, y1), class_name, fill="white", font=font)

    return image


# === Grid-Erstellung ===
for i in range(0, len(selected_images), grid_size):
    subset = selected_images[i:i + grid_size]
    images = []

    for img_path in subset:
        img = Image.open(img_path).convert("RGB")
        base_name = os.path.splitext(os.path.basename(img_path))[0]
        label_path = os.path.join(label_dir, base_name + ".txt")
        img = draw_yolo_boxes(img, label_path)
        images.append(img)

    cell_w, cell_h = images[0].size
    grid_img = Image.new("RGB", (grid_cols * cell_w, grid_rows * cell_h))

    for idx, img in enumerate(images):
        row = idx // grid_cols
        col = idx % grid_cols
        grid_img.paste(img, (col * cell_w, row * cell_h))

    out_path = os.path.join(output_dir, f"grid_{i // grid_size + 1}.png")
    grid_img.save(out_path)
    print(f"Gespeichert: {out_path}")


Gespeichert: visualized_labels_old_dataset/grid_1.png
Gespeichert: visualized_labels_old_dataset/grid_2.png
Gespeichert: visualized_labels_old_dataset/grid_3.png
Gespeichert: visualized_labels_old_dataset/grid_4.png
Gespeichert: visualized_labels_old_dataset/grid_5.png
Gespeichert: visualized_labels_old_dataset/grid_6.png
Gespeichert: visualized_labels_old_dataset/grid_7.png
Gespeichert: visualized_labels_old_dataset/grid_8.png
Gespeichert: visualized_labels_old_dataset/grid_9.png
Gespeichert: visualized_labels_old_dataset/grid_10.png
